In [1]:
from pathlib import Path

P = Path("D:/Dokumente/")
# P = Path("D:/Bilder/")


In [2]:
for _ in P.rglob("*"):
    pass

In [3]:
def elem_type(p):
    if p.is_file():
        return "file"
    elif p.is_dir():
        return "dir"
    else:
        return "other"

def _analyse_folder(p):
    for elem in p.iterdir():
        try:
            t = elem_type(elem)
        except OSError:
            continue

        if t == "file":
            pass
        elif t == "dir":
            _analyse_folder(elem)

In [4]:
_analyse_folder(P)

In [ ]:
source = Path("D:/Dokumente/")
target = Path("D:/Dokumente2/")

source_paths = set(source.rglob("*"))
target_paths = set(target.rglob("*"))
source_paths = {p.relative_to(source) for p in source_paths}
target_paths = {p.relative_to(target) for p in target_paths}

# get all relative paths for files and folders which are in target but not in source
paths_only_in_target = target_paths - source_paths
removed_files = {p for p in paths_only_in_target if (target / p).is_file()}
removed_folders = {p for p in paths_only_in_target if (target / p).is_dir()}
removed_toplevel_folders = {
    p for p in removed_folders if p.parent not in removed_folders
}

# get all relative paths for files and folders which are in source but not in target
paths_only_in_source = source_paths - target_paths
new_files = {p for p in paths_only_in_source if (source / p).is_file()}
new_folders = {p for p in paths_only_in_source if (source / p).is_dir()}
new_toplevel_folders = {
    p for p in new_folders if p.parent not in new_folders
}

#
paths_in_source_and_target = source_paths & target_paths
changed_files = {}
changed_folder2file = {}
changed_file2folder = {}
unchanged_files = {}
for p in paths_in_source_and_target:
    if (source / p).is_file() and (target / p).is_file():
        if filecmp(source / p, target / p):
            changed_files.add(p)
        else:
            unchanged_files.add(p)
    elif source / p.is_file() and target / p.is_dir():
        changed_file2folder.add(p)
    elif source / p.is_dir() and target / p.is_file():
        changed_folder2file.add(p)
    

In [5]:
import uuid

s1 = set([uuid.uuid4() for _ in range(300_000)])